In [1]:
import numpy as np
from sklearn.metrics import jaccard_similarity_score

In [2]:
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("data/comoda_data.csv")

In [5]:
df = df.replace(-1.0, np.nan)
df = df.dropna(how="any")

## Clean Data Here

In [6]:
## CONSTANTS ASSUMPTION
sigma = 1
threshold = 0.1
columns = df.columns.tolist()
columns.remove("rating")
columns.append("rating")
df = df[columns]
X = df[columns[2:-1]]
y = df[columns[-1:]]
df = df.reset_index(drop=True)
#df

In [7]:
class neighbors:
    def __init__(self, X, y):
        self.model = KNeighborsClassifier(n_neighbors=10)
        self.model.fit(X,y)
    def get_neighbors_data(self, user, itemID):
        #all_neighbors_indices = self.model.kneighbors(user[2:-1])[1][0]
        #print(all_neighbors_indices)
        #neighbors_data = df.iloc[all_neighbors_indices]
        neighbors_data = df[(df.itemID == itemID)]
        #print(neighbors_data)
        self.good_neighbors = pd.DataFrame()
        #print(neighbors_data.shape)
        threshold = 0.3
        for index, row in neighbors_data.iterrows():
            similarity = jaccard_similarity_score(row[2:-1], user[2:-1])
            #print(similarity)
            if similarity > threshold:
                self.good_neighbors=self.good_neighbors.append(row, ignore_index=True)
                #print(row)
                #print(good_neighbors)
                #print("Inside Threshold")
            #print(self.good_neighbors)
        #print(self.good_neighbors)
        return self.good_neighbors

In [8]:
class similarity_class:
    def __init__(self):
        pass
    def jaccard_similarity(self, x, y):
        return jaccard_similarity_score(x,y)

In [9]:
class neighborhood_contribution:
    def __init__(self):
        pass
    def neighbor_rating(self, neighbor, itemID, sigma, threshold):
        self.ratings_sum=0
        self.similarity_sum=0
        #print(type(user))
        #print(user)
        #print(neighbor[2:10])
        #print(df.itemID)
        ratings = df[(df.userID == neighbor.userID) & (df.itemID == itemID)]
        #print(ratings.shape)
        for index, user_rating in ratings.iterrows():
            similarity = jaccard_similarity_score(neighbor[2:-1],user_rating[2:-1])
            #print(similarity)
            if similarity > threshold:
                self.ratings_sum += user_rating.rating * similarity
                self.similarity_sum += similarity
                #print(self.similarity_sum)
            #print(self.ratings_sum)
            #print(neighbor[-10:],user_rating[-10:])
        #print("Rating sum")
        #print(self.ratings_sum)
        #print("Similarity Sum")
        #print(self.similarity_sum)
        try:
            rating = self.ratings_sum / self.similarity_sum
            #rating = ratings.rating.mean()
        except:
            rating = 0
        #print(rating)
        return rating
    def neighbor_average(self,neighbor,sigma,threshold):
        ratings_of_neighbor = df[df.userID == neighbor.userID]#.rating.mean()
        rating_sum=0;
        count=0;
        for index, row in ratings_of_neighbor.iterrows():
            similarity = jaccard_similarity_score(row[2:-1], neighbor[2:-1])
            if similarity > threshold:
                rating_sum+=row.rating
                count+=1
        average_rating_in_given_context = rating_sum/count
        #print(average_rating_in_given_context)
        return average_rating_in_given_context
    def baseline_rating(self, itemID):
        average_item_rating = df[df.itemID == itemID].rating.mean()
        return average_item_rating

In [10]:
def numerator(neighborhood_contribution_rating, similarity_of_neighbors):
    numerator_sum=0
    #print(neighborhood_contribution_rating)
    #print(similarity_of_neighbors)
    for neighbor_rating, similarity in zip(neighborhood_contribution_rating,similarity_of_neighbors):
        numerator_sum+= neighbor_rating*similarity
    return numerator_sum

In [11]:
def predict_rating(user,itemID):
    similarity_function = similarity_class()
    user_neighbors = neighbors(X,y)
    user_neighborhood_contribution = neighborhood_contribution()
    neighbors_data = user_neighbors.get_neighbors_data(user,itemID)
    #print(neighbors_data.shape)
    #print(neighbors_data)
    #similarity_of_neighbors = [similarity_function.jaccard_similarity(neighbor[2:-1],user[2:-1]) for neighbor in neighbors_data]
    similarity_of_neighbors = []
    neighborhood_contribution_rating = []
    #print(neighbors_data.shape)
    for index, neighbor in neighbors_data.iterrows():
        similarity_of_neighbors.append(similarity_function.jaccard_similarity(neighbor[2:-1],user[2:-1]))
        neighborhood_contribution_rating.append(user_neighborhood_contribution.neighbor_rating(neighbor,itemID,1,0.1) - user_neighborhood_contribution.neighbor_average(neighbor,1,0.1))
#     neighborhood_contribution_rating = [user_neighborhood_contribution.neighbor_rating(neighbor,item) - user_neighborhood_contribution.neighbor_average(neighbor)
#                                         for neighbor in neighbors_data]
    numerator_value = numerator(neighborhood_contribution_rating, similarity_of_neighbors)
    baseline_rating = user_neighborhood_contribution.baseline_rating(itemID)
    #print(numerator_value)
    #print(similarity_of_neighbors)
    #print(baseline_rating)
    final_rating = baseline_rating + numerator_value/ sum(similarity_of_neighbors)
    return final_rating

In [12]:
# if __name__ == "main":
#     userId = 20
#     iteId = 30
#     rating = predict_rating(20,30)
#     print(rating)

In [13]:
predict_rating(df.iloc[37], 47)
#df.iloc[5]"

5.0479068684211841

In [14]:
#df[df.itemID == 47]

In [15]:
#df.iloc[37]

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
#y_true = [df.iloc[i].rating for i in range(700,800)]

In [18]:
#y_pred = [predict_rating(df.iloc[i], df.iloc[i].itemID) for i in range(700,800)]

In [19]:
#y_pred

In [20]:
#mean_squared_error(y_true, y_pred)

## Pyswarm Implementation

In [21]:
# from pyswarm import pso

# def banana(x):
#     x1 = x[0]
#     x2 = x[1]
#     return x1**4 - 2*x2*x1**2 + x2**2 + x1**2 - 2*x1 + 5

# def con(x):
#     x1 = x[0]
#     x2 = x[1]
#     return [-(x1 + 0.25)**2 + 0.75*x2]

# lb = [-3, -1]
# ub = [2, 6]

# xopt, fopt = pso(banana, lb, ub, f_ieqcons=con)

In [22]:
from pyswarm import pso

In [23]:
len(df.rating.unique())

5

In [24]:
columns = df.columns

In [25]:
columns = columns[2:-1]

In [26]:
columns

Index(['age', 'sex', 'city', 'country', 'time', 'daytype', 'season',
       'location', 'weather', 'social', 'endEmo', 'dominantEmo', 'mood',
       'physical', 'decision', 'interaction', 'director', 'movieCountry',
       'movieLanguage', 'movieYear', 'genre1', 'genre2', 'genre3', 'actor1',
       'actor2', 'actor3', 'budget'],
      dtype='object')

In [27]:
from collections import defaultdict
size_of_feature = defaultdict(dict)
for column in columns:
    size_of_feature[column]['length'] = len(df[column].unique())
    size_of_feature[column]['names'] = df[column].unique()
    

In [28]:
#size_of_feature

In [29]:
#size_of_feature['actor1']

{'length': 329,
 'names': array([ 1636.,  1373.,  1382.,    98.,   384.,   862.,  1803.,  1635.,
          595.,    74.,  1877.,   817.,  1126.,   251.,  1679.,  1957.,
          223.,   691.,   719.,   494.,   491.,  1180.,   526.,  1347.,
          230.,  1169.,   869.,   190.,   749.,  1420.,  1344.,   175.,
         1564.,   476.,    20.,  1439.,   437.,     6.,  1953.,   951.,
          349.,    16.,  1478.,  1241.,  1121.,   357.,   115.,   290.,
         1659.,   926.,  1630.,  1212.,   136.,  1919.,   385.,   833.,
         1863.,  1103.,   925.,  1622.,  1025.,   543.,   710.,   231.,
         1098.,  1437.,  1731.,  1794.,   271.,   541.,   660.,   699.,
          507.,   426.,  1331.,   973.,   881.,  1871.,   414.,  1592.,
          998.,  1638.,  1495.,   747.,  1696.,  1227.,  1611.,   475.,
          665.,   969.,   139.,  1688.,  1335.,   911.,  1866.,   531.,
         1884.,   807.,  1483.,  1685.,  1947.,  1576.,   895.,  1680.,
          704.,  1857.,   506.,  1859.,

In [30]:
df.actor1.max()

1994.0

In [31]:
df.actor1.describe()

count    1114.000000
mean     1012.370736
std       568.663979
min         4.000000
25%       531.000000
50%       969.000000
75%      1518.250000
max      1994.000000
Name: actor1, dtype: float64

In [32]:
feature_dict = defaultdict(dict)
for k,v in size_of_feature.items():
    for i,j in v.items():
        for name in v['names']:
            feature_dict[k][name] = sum(df[k] == name)
            #print(k,name)

In [33]:
#feature_dict

In [34]:
#df[df.columns[2:]].corr(method='kendall', min_periods=1)

In [43]:
df.columns

Index(['userID', 'itemID', 'age', 'sex', 'city', 'country', 'time', 'daytype',
       'season', 'location', 'weather', 'social', 'endEmo', 'dominantEmo',
       'mood', 'physical', 'decision', 'interaction', 'director',
       'movieCountry', 'movieLanguage', 'movieYear', 'genre1', 'genre2',
       'genre3', 'actor1', 'actor2', 'actor3', 'budget', 'rating'],
      dtype='object')

In [64]:
df_new = df.drop(['director','genre1', 'genre2',
       'genre3', 'actor1', 'actor2', 'actor3','budget'], axis=1,)

In [65]:
df_new.describe()

,userID,itemID,age,sex,city,country,time,daytype,season,location,...,endEmo,dominantEmo,mood,physical,decision,interaction,movieCountry,movieLanguage,movieYear,rating
count,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,...,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000
mean,60.482047,2541.308797,28.022442,1.470377,14.984740,2.623878,2.892280,1.698384,2.859964,1.201975,...,4.139138,4.424596,1.578097,1.103232,1.263016,1.214542,35.125673,9.280072,2005.216338,3.834829
std,50.987923,1813.884533,6.119831,0.499346,5.831625,0.836824,0.850492,0.643341,1.059825,0.549079,...,2.370346,2.227446,0.597769,0.304398,0.440469,0.410689,6.522778,1.955621,8.955057,1.012770
min,15.000000,1.000000,15.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1902.000000,1.000000
25%,31.000000,175.000000,26.000000,1.000000,10.000000,2.000000,2.000000,1.000000,2.000000,1.000000,...,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,37.000000,9.000000,2004.000000,3.000000
50%,50.000000,3647.000000,28.000000,1.000000,20.000000,2.000000,3.000000,2.000000,3.000000,1.000000,...,4.000000,4.000000,2.000000,1.000000,1.000000,1.000000,37.000000,9.000000,2009.000000,4.000000
75%,59.000000,3991.750000,30.000000,2.000000,20.000000,3.000000,3.000000,2.000000,4.000000,1.000000,...,7.000000,7.000000,2.000000,1.000000,2.000000,1.000000,37.000000,9.000000,2010.000000,5.000000
max,268.000000,4378.000000,63.000000,2.000000,22.000000,5.000000,4.000000,3.000000,4.000000,3.000000,...,7.000000,7.000000,3.000000,2.000000,2.000000,2.000000,37.000000,27.000000,2012.000000,5.000000


In [66]:
new_size_of_feature = defaultdict(dict)
for column in df_new.columns:
    new_size_of_feature[column]['length'] = len(df_new[column].unique())
    new_size_of_feature[column]['names'] = df_new[column].unique()

In [67]:
sum_of_features = 0
for k,v in new_size_of_feature.items():
    sum_of_features += v['length']

In [68]:
sum_of_features

855

In [69]:
df_new.columns

Index(['userID', 'itemID', 'age', 'sex', 'city', 'country', 'time', 'daytype',
       'season', 'location', 'weather', 'social', 'endEmo', 'dominantEmo',
       'mood', 'physical', 'decision', 'interaction', 'movieCountry',
       'movieLanguage', 'movieYear', 'rating'],
      dtype='object')

In [70]:
df_age = pd.get_dummies(df.age)

In [72]:
df_age.head()

,15.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,...,27.0,28.0,29.0,30.0,31.0,33.0,34.0,35.0,36.0,63.0
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [81]:
for column in df_new.columns[2:]:
    df_col = pd.get_dummies(df_new[column])
    df_new = pd.concat([df_new, df_col], axis=1)

KeyboardInterrupt: 

In [ ]:
df_new.drop(['age', 'sex', 'city', 'country', 'time', 'daytype',
       'season', 'location', 'weather', 'social', 'endEmo', 'dominantEmo',
       'mood', 'physical', 'decision', 'interaction', 'movieCountry',
       'movieLanguage', 'movieYear', 'rating'], axis=1)